# Install Package

In [ ]:
!pip install PyPDF2
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


# Import Library

In [ ]:
import pandas as pd
import PyPDF2
from google.colab import files
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from sklearn.metrics import jaccard_score

# Step 1: Upload Files

In [ ]:
uploaded = files.upload()

valid_skills = {
    "python", "java", "c++", "c#", "r", "kotlin", "javascript", "php", "html", "css", "ruby",
    "tensorflow", "keras", "scikit-learn", "pandas", "numpy", "matplotlib", "seaborn", "django", "flask", "spring",
    "sql", "mysql", "postgresql", "microsoft sql", "nosql", "mongodb", "apache spark", "hadoop", "power bi", "tableau", "excel",
    "google cloud platform", "aws", "machine learning", "deep learning", "neural networks", "natural language processing",
    "nlp", "computer vision", "random forest", "decision trees", "regression", "classification", "clustering",
    "big data", "google bigquery", "kubernetes", "docker", "heroku", "azure", "time series analysis", "anova", "statistics",
    "data mining", "data analysis", "raspberry pi", "automation", "git", "github", "jira", "restapi", "selenium",
    "ci/cd pipelines", "jenkins", "tableau","apache kafka", "apache flink", "airflow", "luigi", "etl",
    "google kubernetes engine", "amazon elastic kubernetes service", "microsoft azure functions",
    "terraform", "ansible", "chef", "puppet", "vagrant",
    "plotly", "d3.js", "power bi", "qlikview",
    "bitbucket", "gitlab",
    "amazon s3", "google cloud storage", "azure blob storage",
    "apache hadoop", "apache cassandra", "apache hive", "apache pig",
    "swift", "rust", "go",
    "node.js", "express.js", "vue.js", "angular",
    "reinforcement learning", "generative adversarial networks", "gan", "bayesian networks",
    "mqtt", "zigbee", "lorawan", "ble", "bluetooth low energy",
    "ethereum", "hyperledger", "smart contracts", "solidity",
    "ethical hacking", "penetration testing", "siem", "firewalls", "vpn",
    "flutter", "react native", "swift", "android sdk",
    "sap businessobjects", "microstrategy", "sas business analytics",
    "unity", "unreal engine", "cocos2d",
    "opencv", "yolo", "mask r-cnn", "image segmentation"
}

Saving candidate_000.pdf to candidate_000.pdf
Saving candidate_001.pdf to candidate_001.pdf
Saving candidate_002.pdf to candidate_002.pdf
Saving candidate_003.pdf to candidate_003.pdf
Saving candidate_004.pdf to candidate_004.pdf
Saving candidate_005.pdf to candidate_005.pdf
Saving candidate_006.pdf to candidate_006.pdf
Saving candidate_007.pdf to candidate_007.pdf
Saving candidate_008.pdf to candidate_008.pdf
Saving candidate_009.pdf to candidate_009.pdf
Saving candidate_010.pdf to candidate_010.pdf
Saving candidate_011.pdf to candidate_011.pdf
Saving candidate_012.pdf to candidate_012.pdf
Saving candidate_013.pdf to candidate_013.pdf
Saving candidate_014.pdf to candidate_014.pdf
Saving candidate_015.pdf to candidate_015.pdf
Saving candidate_016.pdf to candidate_016.pdf
Saving candidate_017.pdf to candidate_017.pdf
Saving candidate_018.pdf to candidate_018.pdf
Saving candidate_019.pdf to candidate_019.pdf
Saving candidate_020.pdf to candidate_020.pdf
Saving candidate_021.pdf to candid

# Step 2: Extract Text from PDF

In [ ]:
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, "rb") as f:
            pdf_reader = PyPDF2.PdfReader(f)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text
    except:
        return ""  # Return empty string if any error occurs

# Step 3: Extract Skills from PDF

In [ ]:
def extract_skills_from_text(text, keyword_list):
    words = text.lower().split()  # Tokenize text by spaces and convert to lowercase
    found_skills = [word for word in words if word in keyword_list]
    return list(set(found_skills))  # Return unique skills

# Step 4: Process Uploaded CVs and Add New Skills to valid_skills

In [ ]:
cv_data = []
updated = True  # Flag to check if any update in skills happened

while updated:  # Continue looping if any new skills were added
    updated = False  # Reset the flag for this loop
    for filename in uploaded:
        if filename.endswith(".pdf"):
            cv_text = extract_text_from_pdf(filename)  # Extract text from the PDF file
            if cv_text:
                skills = extract_skills_from_text(cv_text, valid_skills)  # Extract skills from the text
                cv_data.append({"filename": filename, "skills": skills})

                # Add any new skills found in the CV to valid_skills
                new_skills = [skill for skill in skills if skill not in valid_skills]
                if new_skills:  # If there are new skills
                    valid_skills.update(new_skills)  # Add new skills to valid_skills
                    updated = True  # Set the flag to True if new skills were added
                    break  # Exit the loop to re-process all CVs again

# Convert CV data to DataFrame
df_cv = pd.DataFrame(cv_data)

# Display DataFrame with extracted skills
print(df_cv)

# Display updated valid_skills
print("\nUpdated valid skills:", valid_skills)

              filename                                    skills
0    candidate_000.pdf        [classification, pandas, excel, r]
1    candidate_001.pdf                             [nlp, hadoop]
2    candidate_002.pdf                           [flask, python]
3    candidate_003.pdf                                    [mqtt]
4    candidate_004.pdf                                        []
..                 ...                                       ...
145  candidate_145.pdf                                        []
146  candidate_146.pdf                       [classification, r]
147  candidate_147.pdf                       [classification, r]
148  candidate_148.pdf  [classification, clustering, regression]
149  candidate_149.pdf                              [tensorflow]

[150 rows x 2 columns]

Updated valid skills: {'anova', 'sql', 'decision trees', 'statistics', 'docker', 'smart contracts', 'postgresql', 'data mining', 'php', 'vagrant', 'mqtt', 'natural language processing', 'apache s

In [ ]:
# Save the DataFrame to a JSON file
df_cv.to_json("cv_data.json", orient="records", lines=True)

print("DataFrame has been saved as cv_data.json")


DataFrame has been saved as cv_data.json


# Step 5: Process Job List

In [ ]:
# Load job list
df_jobs = pd.read_csv('job_list.csv')

# Keywords for job titles and corresponding skills
job_keywords = {
    "data": ["python", "sql", "data analysis", "statistics", "excel", "power bi", "tableau"],
    "analyst": ["sql", "python", "data visualization", "business analysis", "statistics"],
    "engineering": ["python", "sql", "java", "c++", "cloud computing", "docker", "kubernetes"],
    "manager": ["project management", "excel", "jira", "leadership", "communication"],
    "designer": ["photoshop", "illustrator", "ui/ux", "adobe xd", "figma"],
    "electrical": ["circuit design", "matlab", "simulink", "power systems"],
    "illustrator": ["illustrator", "photoshop", "adobe suite"],
    "application": ["application development", "java", "android", "ios", "flutter"],
    "developer": ["java", "javascript", "python", "html", "css", "react", "node.js"],
    "business": ["business analysis", "strategy", "excel", "data analysis", "communication"]
}

# Function to generate skills_needed based on job_title
def generate_skills_needed(job_title):
    skills_needed = []
    for keyword, skills in job_keywords.items():
        if keyword in job_title.lower():
            skills_needed.extend(skills)
    return list(set(skills_needed))

df_jobs["skills_needed"] = df_jobs["job_title"].apply(generate_skills_needed)

In [ ]:
# Function to generate skills_needed based on job_title
def generate_skills_needed(job_title):
    skills_needed = []
    for keyword, skills in job_keywords.items():
        if keyword in job_title.lower():
            skills_needed.extend(skills)
    return list(set(skills_needed))

df_jobs["skills_needed"] = df_jobs["job_title"].apply(generate_skills_needed)

# Step 6: Pre-Process for TensorFlow Model

In [ ]:
# Initialize MultiLabelBinarizer for encoding skills
mlb = MultiLabelBinarizer()

# Transform the skills in CVs and job requirements into binary format
cv_skills_matrix = mlb.fit_transform(df_cv["skills"].values)
job_skills_matrix = mlb.fit_transform(df_jobs["skills_needed"].values)

# Generate training data (cartesian product of CVs and jobs)
X = []  # Features
y = []  # Labels

for cv_idx, cv_skills in enumerate(cv_skills_matrix):
    for job_idx, job_skills in enumerate(job_skills_matrix):
        # Combine CV and job skills into a single feature vector
        X.append(np.concatenate([cv_skills, job_skills]))  # Combine CV and job skill features

        # Label as 1 if there's a significant overlap between CV and job skills, otherwise 0
        overlap_count = len(set(cv_skills).intersection(set(job_skills)))  # Count the overlap
        y.append(1 if overlap_count > 0 else 0)  # If overlap, label 1, else 0

# Convert X and y to numpy arrays
X = np.array(X)
y = np.array(y)

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Build and Train the Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(2, activation="relu", input_shape=(X.shape[1],)),  # Hidden layer with 2 neurons
    tf.keras.layers.Dropout(0.2),  # Dropout for regularization to reduce overfitting
    tf.keras.layers.Dense(1, activation="sigmoid")  # Output layer: binary classification (1 or 0)
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# EarlyStopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

# Train the model with the training data, using validation data
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model accuracy: {accuracy:.2f}")

Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


672/672 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9410 - loss: 0.4204 - val_accuracy: 1.0000 - val_loss: 0.0481
Epoch 2/3
672/672 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.1167 - val_accuracy: 1.0000 - val_loss: 0.0165
Epoch 3/3
672/672 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 0.0770 - val_accuracy: 1.0000 - val_loss: 0.0074
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0073
Model accuracy: 1.00


# Step 8: Generate Job Recommendations Using Cosine Similarity

In [ ]:
recommendations = []

for _, cv_row in df_cv.iterrows():
    # Mengubah keterampilan kandidat menjadi format yang sesuai dengan MultiLabelBinarizer
    candidate_skills = mlb.transform([cv_row["skills"]])[0]
    scores = []

    for job_idx, job_skills in enumerate(job_skills_matrix):
        # Menghitung cosine similarity antara keterampilan kandidat dan keterampilan pekerjaan
        similarity = cosine_similarity([candidate_skills], [job_skills])[0][0]
        scores.append((job_idx, similarity))

    # Mendapatkan 3 pekerjaan teratas berdasarkan kesamaan (similarity)
    top_3_jobs = sorted(scores, key=lambda x: x[1], reverse=True)[:3]
    recommendations.append({
        "filename": cv_row["filename"],
        "recommendations": [
            {
                "job_title": df_jobs.loc[job[0], "job_title"],
                "job_link": df_jobs.loc[job[0], "job_link"],
                "similarity": job[1]
            }
            for job in top_3_jobs
        ]
    })

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['classification', 'pandas', 'r'] will be ignored
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['hadoop', 'nlp'] will be ignored
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['flask'] will be ignored
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['mqtt'] will be ignored
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['aws', 'r'] will be ignored
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['gan'] will be ignored
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/

# Step 9: Display Results

In [ ]:
expanded_recommendations = []

for row in recommendations:
    recs = row["recommendations"]
    filename = row["filename"]
    rec_data = {"filename": filename}

    # Ambil 3 rekomendasi teratas dan pisahkan ke dalam kolom yang berbeda
    for i, rec in enumerate(recs[:3]):  # Hanya mengambil 3 rekomendasi teratas
        rec_data[f"recommendation_{i+1}"] = rec["job_title"]
        rec_data[f"link_{i+1}"] = rec["job_link"]
        rec_data[f"similarity_{i+1}"] = rec["similarity"]

    expanded_recommendations.append(rec_data)

df_expanded = pd.DataFrame(expanded_recommendations)
print(df_expanded.head())

            filename                                   recommendation_1  \
0  candidate_000.pdf                             Outage Manager - Texas   
1  candidate_001.pdf  Sr. Principal Analyst – Data Analytics & AI in...   
2  candidate_002.pdf                System Analytics - Pharmacy Analyst   
3  candidate_003.pdf  Sr. Principal Analyst – Data Analytics & AI in...   
4  candidate_004.pdf  Sr. Principal Analyst – Data Analytics & AI in...   

                                              link_1  similarity_1  \
0  https://www.indeed.com/rc/clk?jk=0209edfacd6ac...      0.447214   
1  https://www.indeed.com/rc/clk?jk=f2ee679edfca4...      0.000000   
2  https://www.indeed.com/rc/clk?jk=874d8ef92e7ed...      0.447214   
3  https://www.indeed.com/rc/clk?jk=f2ee679edfca4...      0.000000   
4  https://www.indeed.com/rc/clk?jk=f2ee679edfca4...      0.000000   

                             recommendation_2  \
0  Technical Service Support Manager - Remote   
1         System Analytics - P

In [ ]:
# Save df_jobs to a JSON file
df_jobs.to_json("jobs_data.json", orient="records", lines=True)

print("DataFrame has been saved as jobs_data.json")


DataFrame has been saved as jobs_data.json


In [ ]:
# Save the model to an .h5 file
model.save("model_binary_classification.h5")
print("Model has been saved as model_binary_classification.h5")


Model has been saved as model_binary_classification.h5


# **BUILD REQUIREMENT.TXT**

In [ ]:
# Create the requirements.txt file manually
with open('requirements.txt', 'w') as f:
    f.write("pandas\n")
    f.write("PyPDF2\n")
    f.write("google-colab\n")
    f.write("scikit-learn\n")
    f.write("tensorflow\n")
    f.write("transformers\n")

# Download the requirements.txt file to your computer
from google.colab import files
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>